In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/data/project03/gpt

/content/drive/MyDrive/data/project03/gpt


In [ ]:
!pip install transformers
!pip install datasets

In [4]:
import pandas as pd
import numpy as np
import re

In [5]:
# 훈련모델
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer = tokenizer,
        mlm = mlm,
    )
    return data_collator

def train(train_file_path, model_name, output_dir, overwrite_output_dir,
          per_device_train_batch_size, num_train_epochs, save_steps):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
                bos_token = '</s>', eos_token = '</s>', unk_token = '<unk>',
                pad_token = '<pad>', mask_token = '<mask>')
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir, legacy_format = False)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir = output_dir,
        overwrite_output_dir = overwrite_output_dir,
        per_device_eval_batch_size = per_device_train_batch_size,
        num_train_epochs = num_train_epochs,
    )

    trainer = Trainer(
        model = model,
        args = training_args,
        data_collator = data_collator,
        train_dataset = train_dataset,
    )

    trainer.train()
    trainer.save_model()

# 실행 모델
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

def load_model(model_path):
  model = GPT2LMHeadModel.from_pretrained(model_path)
  return model

def load_tokenizer(tokenizer_path):
  tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
  return tokenizer

def generate_text(sequence, max_lenth):
  model_path = '/content/drive/MyDrive/data/project03/gpt/results'
  model = load_model(model_path)
  tokenizer = load_tokenizer(model_path)
  ids = tokenizer.encode(f'{sequence},', return_tensors = 'pt')
  final_outputs = model.generate(
      ids,
      do_sample = True,
      max_length = max_length,
      pad_token_id = model.config.pad_token_id,
      tok_k = 30, # 가장 높은 확률을 지닌 n개의 단어수 중 추출
      top_p = 0.95 # 누적확률이 n%인 단어까지 포함하여 추출
      # repetition_penalty = 10 # 단어 빈도 패널티
  )
  result = tokenizer.decode(final_outputs[0], skip_special_tokens = True)
  return result

In [ ]:
# 훈련
# 훈련 파일 경로 설정
train_file_path = '/content/drive/MyDrive/data/project03/gpt/music/melon_rap_lyrics.csv'
model_name = 'skt/kogpt2-base-v2'
output_dir = '/content/drive/MyDrive/data/project03/gpt/results'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 1.5
save_steps = 500

train(train_file_path = train_file_path,
      model_name = model_name,
      output_dir = output_dir,
      overwrite_output_dir = overwrite_output_dir,
      per_device_train_batch_size = per_device_train_batch_size,
      num_train_epochs = num_train_epochs,
      save_steps = save_steps
)


Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

***** Running training *****
  Num examples = 1968
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 369


Step,Training Loss


In [ ]:
# 실행
# 셋팅 tok_k = 50, top_p = 0.95
input = '남자 여자 해변 여유로움'
sequence = input
max_length = 128 
print('input :' + sequence)
generate_text(sequence, max_length)

loading configuration file /content/drive/MyDrive/data/project03/gpt/results/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_la

input :남자 여자 해변 여유로움


All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at /content/drive/MyDrive/data/project03/gpt/results.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file /content/drive/MyDrive/data/project03/gpt/results/added_tokens.json. We won't load it.
loading file None
loading file /content/drive/MyDrive/data/project03/gpt/results/special_tokens_map.json
loading file /content/drive/MyDrive/data/project03/gpt/results/tokenizer_config.json
loading file /content/drive/MyDrive/data/project03/gpt/results/tokenizer.json


남자 여자 해변 여유로움, 넉넉한 여름  
하늘이 높자 나비는 꽃처럼 하늘 품에 안느라 날으는 둥 마는 둥  하늘은 온통 하얀색  나비는 꽃처럼 하늘 품에 안느라 날으는 둥  
"바람이 많다.  부는 바람이 너무 크다.  오는 비도 고울 뿐이다.  꽃도 많고 나무도 많고 꽃나무도 많다.  저 태울 것 없다.  "
"이불 한 덮이에 누운 손목이 반짝이다.  소나기에 비를 모아 덮었던 것 같아  그 손끝은 종일 고갤 적중했다.


In [ ]:
# 실행
# tok_k = 1, top_p = 0.9, repetition_penalty = 1.5 
input = '남자 여자 해변 여유로움'
sequence = input
max_length = 128 
# print('input :' + sequence)
sentence = generate_text(sequence, max_length)
sentence

loading configuration file /content/drive/MyDrive/data/project03/gpt/results/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_la

남자 여자 해변 여유로움, 넉넉한 여름  
하늘이 높자 나비는 꽃처럼 하늘 향해 날고 새가 하늘을 향한다던 사람이 있었어. 고약한 바람은 여직 푸른 나를 붙들었다. 가시 돋친 듯 활짝은 달 아래 벌벌 부는 바람 그쳐라. 가을이라 더욱 매섭게만 피겠다.  너는 이제껏 한 송이도 없는 것을 저 혼자 더 많이 보았구나, 싶네.’
"바다는 좁습니다. 바람이 너무 세서 걷기도 말리기가 좀 지치신 것 같아요. 내일은 꼭 비켜다오 하고 약속 있어요, 또 놓으소서 하시면 머지않


In [ ]:
# 실행
# tok_k = 10, top_p = 0.9, repetition_penalty = 1.5 
input = '남자 여자 해변 여유로움'
sequence = input
max_length = 128 
# print('input :' + sequence)
sentence = generate_text(sequence, max_length)
sentence

loading configuration file /content/drive/MyDrive/data/project03/gpt/results/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_la

남자 여자 해변 여유로움, 당신 없으면 꼭  나만 보고 싶어.)
하늘이 새로 얻은 하늘, 이 마음대로인 것도 아니고 달도 새 별이라니 나도 어처구니없어. 라며 내민다. 라이터가 제법 잘생겼고 불그레레 청명한 가을 하늘이 마음에 안든다는 말을 들었다. 라면 한 그릇이던 것이 두 컵으로 줄더니 세 줄로나서 백치랑 투닥투닥 붙는다고 했다. 저마다의 버릇에 맞게 좋은 데를 골라 먹도록 하시오."
"여직 네 마음이야 남아 아쉬울 것 없으니 다행이다. 그래도 너그러워


In [ ]:
# 실행
# tok_k = 50, top_p = 0.9, repetition_penalty = 1.5 
input = '남자 여자 해변 여유로움'
sequence = input
max_length = 128 
# print('input :' + sequence)
sentence = generate_text(sequence, max_length)
sentence

loading configuration file /content/drive/MyDrive/data/project03/gpt/results/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_la

남자 여자 해변 여유로움, 넉넉한 여름  
하늘이 높자 나비는 꽃처럼 하늘 향해 날고 새가 하늘을 향한다던 사람이 있었어. 고약한 바람은 여직 푸른 나를 붙들었다. 가시 돋친 듯 활짝은 달 아래 벌벌 부는 바람 그쳐라. 가을이라 더욱 매섭게만 피겠다.  너는 이제껏 한 송이도 없는 것을 저 혼자 더 많이 보았구나, 싶네.’
"바다는 좁습니다. 바람이 너무 세서 걷기도 말리기가 좀 지치신 것 같아요. 내일은 꼭 비켜다오 하고 약속 있어요, 또 놓으소서 하시면 머지않


In [ ]:
# 실행
# tok_k = 50, top_p = 0.9, repetition_penalty = 1 
input = '남자 여자 해변 여유로움'
sequence = input
max_length = 256
# print('input :' + sequence)
sentence = generate_text(sequence, max_length)
sentence

loading configuration file /content/drive/MyDrive/data/project03/gpt/results/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_la

'남자 여자 해변 여유로움, 여유로움  "\n"빗속 다 지나 갑니다.  더는 빗물소리 듣지 말 걸.  여직도 젖은 몸 녹일 곳 못하네.  "\n"여름은 점점 여름대로를 걷고 가을은 가을대로를 걷고   겨울은 점점 가을대로를 걷네.  나는 당신 만큼 가을을 보내고 싶은데  나는 가을을 보내고 싶은데  당신도 가을을 보낸 뒤 가을을 보내면  나는 당신 대로 가을을 보내고 싶은데  \n"오늘은 내가 가장 행복하다는 달인  달력 마지막 장 맨 마지막 줄 끝까지 행복한다는 달력  꼭 봐도 사랑한다는 당신과 나란히 꼭 바라던 달  꼭 봐도 사랑한다는 당신 바라던 달  꼭 보고 사랑한다고  꼭 잊지 않겠습니다.  "\n"달빛에 들뜬 연인을 바라보며  연인 또는 부부 등 뒤로 바쁘게 걸음 옮기는 이들   저들 사이로 빈 가지는 빈 가지 말리느라  연인처럼 발걸음 맞춰 나란히  "\n소란한 여름날  너는 뽀얀 구름  왜 저래 저래 피나부비적인가.  \n여름 한 번 보려고 내 걸음에 제 걸음을 조금 늦추네.  슬며'

In [ ]:
sentence[len(input)+2:].strip('["\n"]')

'여유로움  "\n"빗속 다 지나 갑니다.  더는 빗물소리 듣지 말 걸.  여직도 젖은 몸 녹일 곳 못하네.  "\n"여름은 점점 여름대로를 걷고 가을은 가을대로를 걷고   겨울은 점점 가을대로를 걷네.  나는 당신 만큼 가을을 보내고 싶은데  나는 가을을 보내고 싶은데  당신도 가을을 보낸 뒤 가을을 보내면  나는 당신 대로 가을을 보내고 싶은데  \n"오늘은 내가 가장 행복하다는 달인  달력 마지막 장 맨 마지막 줄 끝까지 행복한다는 달력  꼭 봐도 사랑한다는 당신과 나란히 꼭 바라던 달  꼭 봐도 사랑한다는 당신 바라던 달  꼭 보고 사랑한다고  꼭 잊지 않겠습니다.  "\n"달빛에 들뜬 연인을 바라보며  연인 또는 부부 등 뒤로 바쁘게 걸음 옮기는 이들   저들 사이로 빈 가지는 빈 가지 말리느라  연인처럼 발걸음 맞춰 나란히  "\n소란한 여름날  너는 뽀얀 구름  왜 저래 저래 피나부비적인가.  \n여름 한 번 보려고 내 걸음에 제 걸음을 조금 늦추네.  슬며'

In [ ]:
# 실행
# tok_k = 50, top_p = 0.95, epoch = 10
input = '남자 여자 해변 여유로움'
sequence = input
max_length = 128
# print('input :' + sequence)
sentence = generate_text(sequence, max_length)
sentence[len(input)+2:]

loading configuration file /content/drive/MyDrive/data/project03/gpt/results/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_la

'괜히 맘을 급해 놓고  바로 잡아 채려나, 헤매느라  늦었다, 자책하고 싶다.  "\n"덥다, 덥다며 가는 길옆에 전화가 많다.  걸려오는 전화뿐이라  걸려오는 전화에 죄다 해바라기분하니  더는 전화하지 않아도 된다.  "\n"덥다, 덥다며 가는 길옆에 주차장  세우니  한 손도 못되는 저들끼리 추우더니  거든 말에  겨우내 서성이는 바짝 마른 바람에도 날지 않는 바람  그 바람에 나는 노란 민들레 '

In [ ]:
sentence

'남자 여자 해변 여유로움, 괜히 맘을 급해 놓고  바로 잡아 채려나, 헤매느라  늦었다, 자책하고 싶다.  "\n"덥다, 덥다며 가는 길옆에 전화가 많다.  걸려오는 전화뿐이라  걸려오는 전화에 죄다 해바라기분하니  더는 전화하지 않아도 된다.  "\n"덥다, 덥다며 가는 길옆에 주차장  세우니  한 손도 못되는 저들끼리 추우더니  거든 말에  겨우내 서성이는 바짝 마른 바람에도 날지 않는 바람  그 바람에 나는 노란 민들레 '

In [ ]:
import re
sentence_re = re.sub('"\n', ' ', sentence[len(input)+2:])
sentence_re
tmp = sentence_re.split()
tmp = ' '.join(tmp)
tmp2 = tmp.split('.')
sentence_new = []
sentence_new.extend(tmp2)
sentence_new

['괜히 맘을 급해 놓고 바로 잡아 채려나, 헤매느라 늦었다, 자책하고 싶다',
 ' "덥다, 덥다며 가는 길옆에 전화가 많다',
 ' 걸려오는 전화뿐이라 걸려오는 전화에 죄다 해바라기분하니 더는 전화하지 않아도 된다',
 ' "덥다, 덥다며 가는 길옆에 주차장 세우니 한 손도 못되는 저들끼리 추우더니 거든 말에 겨우내 서성이는 바짝 마른 바람에도 날지 않는 바람 그 바람에 나는 노란 민들레']

In [ ]:
# 실행
# tok_k = 30, top_p = 0.95, repetition_penalty = 0.5, epoch = 10
input = '남자 여자 해변 여유로움'
sequence = input
max_length = 128
# print('input :' + sequence)
sentence = generate_text(sequence, max_length)
sentence_re = re.sub('"\n', ' ', sentence[len(input)+2:])
sentence_re
tmp = sentence_re.split()
tmp = ' '.join(tmp)
tmp2 = tmp.split('.')
sentence_new = []
sentence_new.extend(tmp2)
sentence_new

loading configuration file /content/drive/MyDrive/data/project03/gpt/results/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_la

['여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로움, 여유로']

In [ ]:
# 실행
# tok_k = 30, top_p = 0.95, repetition_penalty = 10, epoch = 5
input = '남자 여자 해변 여유로움'
sequence = input
max_length = 128
# print('input :' + sequence)
sentence = generate_text(sequence, max_length)
sentence_re = re.sub('"\n', ' ', sentence[len(input)+2:])
sentence_re
tmp = sentence_re.split()
tmp = ' '.join(tmp)
tmp2 = tmp.split('.')
sentence_new = []
sentence_new.extend(tmp2)
sentence_new

loading configuration file /content/drive/MyDrive/data/project03/gpt/results/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_la

ValueError: ignored

In [ ]:
# 실행
# tok_k = 30, top_p = 0.95, epoch = 10
input = '남자 여자 해변 여유로움'
sequence = input
max_length = 128
# print('input :' + sequence)
sentence = generate_text(sequence, max_length)
sentence_re = re.sub('"\n', ' ', sentence[len(input)+2:])
sentence_re
tmp = sentence_re.split()
tmp = ' '.join(tmp)
tmp2 = tmp.split('.')
sentence_new = []
sentence_new.extend(tmp2)
sentence_new

loading configuration file /content/drive/MyDrive/data/project03/gpt/results/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_la

['괜히 맘을 급해 놓고 바로 잡아 채려나, 헤매느라 늦었다, 자책하고 싶다',
 ' "덥다, 덥다며 가는 길옆에 전화가 많다',
 ' 걸려오는 전화뿐이라 걸려오는 전화에 죄다 해바라기분하니 더는 전화하지 않아도 된다',
 ' "덥다, 덥다며 가는 길옆에 주차장 세우니 한 손도 못되는 저들끼리 추우더니 거든 말에 겨우내 서성이는 바짝 마른 바람에도 날지 않는 바람 그 바람에 나는 노란 민들레']